In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report


In [2]:
# ANTI-OVERFITTING Configuration
TRAIN_DIR = "data/train"
TEST_DIR = "data/test"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 0.0005  # REDUCED from 0.001
SEED = 42
PREDICTION_THRESHOLD = 0.65

In [4]:
 #Regularization
DROPOUT_1 = 0.3  # Increased from 0.2
DROPOUT_2 = 0.6  # Increased from 0.5
L2_REG = 0.001   

In [5]:
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [6]:
# Load Data

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR, image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    label_mode="binary", seed=SEED
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TEST_DIR, image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    label_mode="binary", shuffle=False, seed=SEED
)
class_names = train_ds.class_names

Found 5232 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


In [7]:
class_names

['NORMAL', 'PNEUMONIA']

In [8]:
# REDUCED Augmentation (less aggressive)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),  # REDUCED from 0.2
    tf.keras.layers.RandomZoom(0.1),      # REDUCED from 0.2
    # Removed RandomContrast
], name="reduced_augmentation")

normalization = tf.keras.layers.Rescaling(1./127.5, offset=-1)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x,y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(lambda x,y: (normalization(x), y), num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(lambda x,y: (normalization(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)


In [9]:
# Class Weights
y_train = np.concatenate([y for x,y in train_ds], axis=0)
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weights_dict = dict(enumerate(class_weights))
print(f"✓ Class weights: {class_weights_dict}")


✓ Class weights: {0: 1.939214232765011, 1: 0.6737058975019315}


In [10]:
# Build Anti-Overfitting Model
print("\nBuilding Anti-Overfitting Model...")
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3), include_top=False, weights="imagenet"
)
base_model.trainable = False

l2_reg = tf.keras.regularizers.l2(L2_REG)

inputs = tf.keras.Input(shape=(224,224,3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)



Building Anti-Overfitting Model...


In [11]:
# Anti-Overfitting Layers
x = tf.keras.layers.BatchNormalization()(x)           # ADDED
x = tf.keras.layers.Dropout(DROPOUT_1)(x)             # INCREASED
x = tf.keras.layers.Dense(64, activation='relu',      # REDUCED from 128
                          kernel_regularizer=l2_reg)(x)
x = tf.keras.layers.BatchNormalization()(x)           # ADDED
x = tf.keras.layers.Dropout(DROPOUT_2)(x)             # INCREASED

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs, name='AntiOverfitModel')

model.compile(
    optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)


In [12]:
print("✓ Model built with:")
print("  + Batch Normalization (2 layers)")
print(f"  + Higher Dropout ({DROPOUT_1}, {DROPOUT_2})")
print("  + Smaller Dense Layer (64 vs 128)")
print(f"  + L2 Regularization ({L2_REG})")
print(f"  + Lower Learning Rate ({LEARNING_RATE})")


✓ Model built with:
  + Batch Normalization (2 layers)
  + Higher Dropout (0.3, 0.6)
  + Smaller Dense Layer (64 vs 128)
  + L2 Regularization (0.001)
  + Lower Learning Rate (0.0005)


In [13]:
# Aggressive Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=3, restore_best_weights=True, verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7, verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model_antioverfit.keras', monitor='val_loss', 
        save_best_only=True, verbose=1
    )
]


In [14]:
# Train
history = model.fit(
    train_ds, validation_data=test_ds, epochs=EPOCHS,
    class_weight=class_weights_dict, callbacks=callbacks, verbose=1
)


Epoch 1/30
164/164 [==============================] - ETA: 0s - loss: 0.5238 - accuracy: 0.7942 - auc: 0.9253
Epoch 1: val_loss improved from inf to 0.38023, saving model to best_model_antioverfit.keras
164/164 [==============================] - 80s 461ms/step - loss: 0.5238 - accuracy: 0.7942 - auc: 0.9253 - val_loss: 0.3802 - val_accuracy: 0.8926 - val_auc: 0.9612 - lr: 5.0000e-04
Epoch 2/30
164/164 [==============================] - ETA: 0s - loss: 0.3662 - accuracy: 0.8792 - auc: 0.9665
Epoch 2: val_loss improved from 0.38023 to 0.33543, saving model to best_model_antioverfit.keras
164/164 [==============================] - 58s 349ms/step - loss: 0.3662 - accuracy: 0.8792 - auc: 0.9665 - val_loss: 0.3354 - val_accuracy: 0.9006 - val_auc: 0.9687 - lr: 5.0000e-04
Epoch 3/30
164/164 [==============================] - ETA: 0s - loss: 0.3246 - accuracy: 0.9077 - auc: 0.9743
Epoch 3: val_loss did not improve from 0.33543
164/164 [==============================] - 55s 330ms/step - loss: 0

In [15]:
# Analyze Overfitting
print("\n" + "="*70)
print("OVERFITTING ANALYSIS")
print("="*70)

train_loss = history.history['loss'][-1]
val_loss = history.history['val_loss'][-1]
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]

loss_gap = val_loss - train_loss
acc_gap = train_acc - val_acc

print(f"\nFinal Results:")
print(f"  Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Gap: {loss_gap:.4f}")
print(f"  Train Acc:  {train_acc:.4f} | Val Acc:  {val_acc:.4f} | Gap: {acc_gap:.4f}")

print(f"\nComparison with Original:")
print(f"  Original Gap: Loss 0.412, Acc 0.120 (12%)")
print(f"  New Gap:      Loss {loss_gap:.3f}, Acc {acc_gap:.3f} ({acc_gap*100:.1f}%)")

if loss_gap < 0.25 and acc_gap < 0.08:
    print("\n✅ OVERFITTING SIGNIFICANTLY REDUCED!")
elif loss_gap < 0.35:
    print("\n⚠️ OVERFITTING IMPROVED BUT STILL MODERATE")
else:
    print("\n❌ OVERFITTING STILL PRESENT")



OVERFITTING ANALYSIS

Final Results:
  Train Loss: 0.2844 | Val Loss: 0.3854 | Gap: 0.1010
  Train Acc:  0.9266 | Val Acc:  0.8670 | Gap: 0.0596

Comparison with Original:
  Original Gap: Loss 0.412, Acc 0.120 (12%)
  New Gap:      Loss 0.101, Acc 0.060 (6.0%)

✅ OVERFITTING SIGNIFICANTLY REDUCED!


In [16]:

# Evaluate
print("\n" + "="*70)
print("EVALUATION")
print("="*70)
results = model.evaluate(test_ds, verbose=0)
print(f"Test Accuracy: {results[1]:.4f} ({results[1]*100:.2f}%)")

# Predictions
predictions = model.predict(test_ds, verbose=0)
true_labels = np.concatenate([y for x,y in test_ds], axis=0).astype(int).flatten()
pred_065 = (predictions > PREDICTION_THRESHOLD).astype(int).flatten()

print(f"\nClassification Report (Threshold {PREDICTION_THRESHOLD}):")
print(classification_report(true_labels, pred_065, 
                          target_names=[str(n) for n in class_names]))



EVALUATION
Test Accuracy: 0.9006 (90.06%)

Classification Report (Threshold 0.65):
              precision    recall  f1-score   support

      NORMAL       0.84      0.94      0.88       234
   PNEUMONIA       0.96      0.89      0.92       390

    accuracy                           0.91       624
   macro avg       0.90      0.91      0.90       624
weighted avg       0.91      0.91      0.91       624



In [17]:
# Save
model.save('final_model_antioverfit.keras')
model.save('final_model_antioverfit.h5')
print("\n✓ Models saved")

print("\n" + "="*70)
print("✅ TRAINING COMPLETE!")
print("="*70)
print("\nFiles created:")
print("  - best_model_antioverfit.keras")
print("  - final_model_antioverfit.keras")
print("  - final_model_antioverfit.h5")



✓ Models saved

✅ TRAINING COMPLETE!

Files created:
  - best_model_antioverfit.keras
  - final_model_antioverfit.keras
  - final_model_antioverfit.h5


/opt/anaconda3/envs/tf_m1/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
